In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

Macro `_latex_std_` created. To execute, type its name (without quotes).
=== Macro contents: ===
get_ipython().run_line_magic('run', 'Latex_macros.ipynb')
 

In [ ]:
# My standard magic !  You will see this in almost all my notebooks.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

%matplotlib inline

In [ ]:
from IPython.display import Image

import cnn_helper
%aimport cnn_helper
cnnh = cnn_helper.CNN_Helper()

# Adversarial examples

Recall the process by which we "invert" an image classifier:

Our optimization becomes
$$
\begin{array}[lll]\\
\x = \argmin{\x} \loss^\ip \\
\text{subject to} \\
\hat{\y}^\ip = \y^{(0)}
\end{array}
$$

which we solved via the derivative of the loss with respect to the inputs
$$
\frac{\partial \loss}{\partial \x}
$$

That is, we are
- starting with $\x = \x^{(0)} $
- modifying $\x$
- to derive an input $\x$ that causes the NN to predict $\hat{y} = \y^{(0)}$ with highest probability

We are inverting the output.

We originally specified initializing the image with with $\x = \x^{(0)}$
where $\x^{(0)}$ was either random noise or all black image.

What would happen if 
- we initialized $\x = \x^{(i')}$
- where $\y^{(0)} \ne \y^{(i')}$ ?

That is: our initial image is from class $\y^{(i')}$ but we give an objective target of $\y^{(0)} \ne \y^{(i')} $

Gradient ascent would create an output
- classified as $\y^{(0)}$
- by modifying an image that is *not* from this class

The $\x$ created is called an *Adversarial Example* as it was intentionally created to cause misclassificatio.

Adversarial examples in action:

<table>
    <tr>
        <center>What class is this ?</center>
    </tr>
    <tr>
        <td><img src="images/cat7_classified.png" width=450></td>
    </tr>
</table>

What about this ?

<table>
    <tr>
        <center>What class is this ?</center>
    </tr>
    <tr>
        <td><img src="images/cat7_as_class_859_classified.png" width=450></td>
    </tr>
</table>

It's almost certainly a toaster !

Your eye can't pick up the difference: that's a real-world problem !

Here is the difference between the two images.

<table>
    <tr>
        <center>Adversarial Cat to Toaster</center>
    </tr>
    <tr>
        <td><img src="images/cat7_classified.png" width=900></td> 
        <td><img src="images/cat7_as_class_859_diff.png" width=900></td>
        <td><img src="images/cat7_as_class_859_classified.png" width=900></td>
    </tr>
</table>

What harm can this do ?

<table>
    <tr>
        <center>Adversarial Stop Sign</center>
    </tr>
    <tr>
        <td><img src="images/adv_stop_sign.jpg" width=800></td> 
    </tr>
</table>

[Robust Physical-World Attacks on Deep Learning Models](https://arxiv.org/abs/1707.08945)

Why does this occur ?

Recall the fundamental assumption of Machine Learning:
- an example from the test set is drawn from the same distribution as the training set

In the case of Adversarial Examples, this condition is not satisfied.

How do we remedy this before I get into a self-driving car ?

- Adversarial training
    - augment training set with adversarial images
    - but attacks are very robust
        - if this is some artifact that can signal fakery
        - adjust your Cost function to penalize for creating the artifact !
        

- Can create adversarial example
    - without manipulating training set
    - without manipulating the trained classifier's weights
    - without access to the classifier !
        - black box versus white box attacks
        
- It turns out that an adversarial example that can fool *several* classifiers
    - is also good at fooling a (time-limited) human !

# Adversarial Reprogramming

We can extend the Gradient Ascent method to perform even bigger tricks:
- Getting a Classifier for Task 1 to do something completely different !

Can we get an ImageNet Classifier to count squares ? Imagenet
- does not have squares as an input image
- or numbers as an output class

This is called *Adversarial Reprogramming*.

<img src="images/adv_reprogram_1.jpg" width=900>

[Adversarial Reprogramming of Neural Networks](https://arxiv.org/abs/1806.11146)

Here's a pictorial to describe the process:
<img src="images/adv_reprogram_2.jpg" width=900>

We refer to our original classifier as solving the Source task.

Our goal is to get the classifier to solve the Target task.

The first issue to address:
- the $(\x^\ip, \y^\ip)$ pairs of the Source task come from a different domain than that of the Target task

$$
\begin{array}[lll]\\
\X_\text{source}, \y_\text{source}: & \text{examples for Source task} \\
\X_\text{target}, \y_\text{target}: &  \text{examples for Target task} \\
\end{array}
$$

We create a simple function $h_f$ to map an $\x \in \X_\text{target}$ to an $\x \in \X_\text{source}$.

This ensures that the input to the Source task is of the right "type".

<img src="images/adv_reprogram_3.jpg" width=900>

$h_f$ simply embeds the Target input into an image, which is the domain of the Source task.

Similarly, we create a function $h_g$ to map the Target label to a Source Label.

This will ensure that the output of the Source task is of the right type.

<img src="images/adv_reprogram_4.jpg" width=900>

Finally, the 
Cost function to optimize

$$
\W = \argmin{\W}{ - \log( \pr{h_g( \y_t ) \; | \;\tilde{\X}_\text{source} }) + \lambda || \W ||^2 }
$$

where
$$
\begin{array}[lll]\\
\tilde{\X}_\text{source} & = & h_f( \W, \X_\text{target}) \\
h_f: & &\y_\text{target} \mapsto \y_\text{source} & \text{map source X to target X} \\
h_g: & &\y_\text{target} \mapsto \y_\text{source} & \text{map source label y to target label} \\\\
\end{array}
$$

- Given an input in the Target domain $\X_\text{target}$
- Transform it into an input $\tilde{\X}_\text{source}$ in the Source domain.
- Use the Source Classifier to predict $\tilde{\X}_\text{source}$ a label in the Source domain
    - The correct label in the Target domain is $\y_t$
    - This maps to label $h_g(\y_t) in the Source domain
    
So we are trying to
- maximize the likelihood that the Source classifier creates the encoding for the correct Target label
- subject to constraining the weights $\W$ (the "frame" into which the Target input is placed)

How do we find the frame $\W$ that "reprograms" the Source Classifier ?

By training it of course !  Just plain old ML.

# Misaligned objectives

We have framed the problem of Deep Learning as one of defining a Cost function that meets your objectives.

This is not as easy as it sound.

Consider the difference between
- "Maximize profit"
- "Maximize profit subject to legal and ethical constraints"

We (hopefully) don't have to state the additional constraints to a human -- we take it for granted.

Not so with a machine that has not been trained with additional objectives.

<img src="images/spider.jpg" width=900>

In [ ]:
print("Done")